## Parameters for all Patients 

In [1]:
from __future__ import division, print_function

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from collections import OrderedDict
from scipy.signal import savgol_filter
from scipy.interpolate import InterpolatedUnivariateSpline

Massage spreadsheet data into a dictionary of patients indexed by integer participant number and with each entry a dataframe with LA and LV volumes indexed as time from trigger in millisecs.

There is also a dataframe of RR and frame intervals intervals indexed on participant number.

In [2]:
volumes = 'FullVolumes.xlsx'
rrintervals = 'RRIntervals.xlsx'

df_vols = pd.read_excel(volumes)
df_rrintervals = pd.read_excel(rrintervals).reset_index(drop=True)
df_rrintervals.index = ['RR_ms']

df_la = df_vols.filter(regex='LA-[0-9]{3,3}$')
df_lv = df_vols.filter(regex='LV-[0-9]{3,3}$')

# map index to integers
df_times = df_rrintervals.filter(regex='LV-[0-9]{3,3}$').T
df_times['PN'] = [int(i[-3:]) for i in df_times.index]
df_times = df_times.set_index('PN')

nrows = df_la.count(axis=0)[0]
df_times['Frame_interval_ms'] = df_times['RR_ms'] / nrows
patients = {}
for patno in [int(col[-3:]) for col in df_la]:
    patients[patno] = pd.concat([
        pd.Series(np.arange(nrows) * df_times['Frame_interval_ms'][patno], name='Time'),
        df_lv['LV-%d' % patno],
        df_la['LA-%d' % patno],
    ], axis=1).set_index('Time')
    patients[patno].columns = ('LV', 'LA')
    patients[patno].pat_name = 'Patient %d' % patno

Parameters of Interest. Work on numpy arrays of volumes without the frame rate so all time values are indices ie in 'frames' rather than millisecs.

In [3]:
def lv_es_index(volumes):
    return np.argmin(volumes)

def lv_split_at_es(volumes):
    es = lv_es_index(volumes)
    return volumes[:es], volumes[es:]

def lv_vol_at_es(volumes):
    return volumes[lv_es_index(volumes)]

def lv_vol_at_ss(volumes):
    return volumes[0]

def lv_vol_at_ed(volumes):
    return volumes[-1]

def idx_nearest(array, value, after=0):
    return np.abs(array[after:]-value).argmin() + after

def lv_idx_for_refill80_full(volumes, use_vss=True):
    vss = lv_vol_at_ss(volumes)
    ves = lv_vol_at_es(volumes)
    ved = lv_vol_at_ed(volumes)
    
    fullvol = vss if use_vss else (vss + ved) / 2
    emptyvol = ves
    
    eighty_vol = 0.2*emptyvol + 0.8*fullvol
    return idx_nearest(volumes, value=eighty_vol, after=lv_es_index(volumes))

def lv_idx_max_systolic_down_slope(volumes, slopes):
    esi = lv_es_index(volumes)
    systolic_down_slopes = -slopes[:esi]
    return systolic_down_slopes.argmax()

def lv_idx_max_recovery_slopes(volumes, slopes):
    esi = lv_es_index(volumes)
    # need to further split into early and late.
    idx80 = lv_idx_for_refill80_full(volumes)
    
    diastolic_early_up_slopes = slopes[esi:idx80]
    diastolic_late_up_slopes = slopes[idx80:]
    idx_early = diastolic_early_up_slopes.argmax() + esi
    idx_late = diastolic_late_up_slopes.argmax() + idx80
    
    return idx_early, idx_late

def la_es_index(volumes):
    return np.argmax(volumes)

def la_split_at_es(volumes):
    es = la_es_index(volumes)
    return volumes[:es], volumes[es:]

def la_vol_at_es(volumes):
    return volumes[la_es_index(volumes)]

def la_vol_at_es(volumes, es):
    return volumes[es]

def la_vol_at_ss(volumes):
    return volumes[0]

def la_vol_at_ed(volumes):
    return volumes[-1]

def la_idx_max_systolic_up_slope(volumes, slopes):
    esi = la_es_index(volumes)
    systolic_down_slopes = slopes[:esi]
    return systolic_down_slopes.argmax()

def la_idx_max_emptying_slopes(volumes, slopes):
    esi = la_es_index(volumes)

    # need to further split into early and late.
    idxsplit = esi + (len(volumes) - esi) // 2
    
    diastolic_early_up_slopes = -slopes[esi:idxsplit]
    diastolic_late_up_slopes = -slopes[idxsplit:]
    idx_early = diastolic_early_up_slopes.argmax() + esi
    idx_late = diastolic_late_up_slopes.argmax() + idxsplit
    
    return idx_early, idx_late


In [4]:
def lv_params(patient, region='LV'):
    units, time_units = 'ml', 'ms'
    patient_name = patient.pat_name
    
    volume_series = patient[region]
    time = np.asarray(volume_series.index)
    volume = np.asarray(volume_series.values) 
    diffs = np.gradient(volume) / np.diff(time)[0]
    r_to_r = time[-1]
    sg_volume     = savgol_filter(x=volume, window_length=5, polyorder=2, mode='interp')
    dt_msecs      = np.mean(np.diff(time))
    # one order higher for the derivative sounds right .. increase window to match
    sg_derivative = savgol_filter(x=volume, window_length=7, polyorder=3, deriv=1, delta=dt_msecs)

    spline = InterpolatedUnivariateSpline(time, sg_volume)
    spline_deriv = InterpolatedUnivariateSpline(time, sg_derivative)

    interpolated_time   = np.linspace(0, time[-1], len(time)*3)
    interpolated_savgol = spline(interpolated_time)
    interpolated_deriv  = spline_deriv(interpolated_time)

    min_vol_index = lv_es_index(interpolated_savgol)
    min_vol_time  = interpolated_time[min_vol_index]
    min_volume    = interpolated_savgol[min_vol_index]
    eighty_percent_idx = lv_idx_for_refill80_full(interpolated_savgol)
    eighty_percent_time = interpolated_time[eighty_percent_idx]
    eighty_percent_vol = interpolated_savgol[eighty_percent_idx]

    max_emptying_index = lv_idx_max_systolic_down_slope(interpolated_savgol, interpolated_deriv)
    max_emptying_time = interpolated_time[max_emptying_index]
    max_emptying_vol = interpolated_savgol[max_emptying_index]
    max_emptying_slope = interpolated_deriv[max_emptying_index]

    max_early_filling_index, max_late_filling_index = lv_idx_max_recovery_slopes(interpolated_savgol, interpolated_deriv)
    max_early_filling_time = interpolated_time[max_early_filling_index]
    max_late_filling_time = interpolated_time[max_late_filling_index]

    max_early_filling_vol = interpolated_savgol[max_early_filling_index]
    max_late_filling_vol = interpolated_savgol[max_late_filling_index]

    max_early_filling_slope = interpolated_deriv[max_early_filling_index]    
    max_late_filling_slope = interpolated_deriv[max_late_filling_index]    
    diastolic_time = r_to_r - min_vol_time
    
    return OrderedDict([
        ('LV_time_end_systole', min_vol_time),
        ('LV_peak_ejection_ml_sec', (-1000 *max_emptying_slope)),
        ('LV_time_peak_ejection',  max_emptying_time),
        ('LV_vol_peak_ejection', max_emptying_vol),
        ('LV_early_peak_filling_ml_sec', (1000 *max_early_filling_slope)),
        ('LV_time_early_peak_filling', (max_early_filling_time - min_vol_time)),
        ('LV_vol_early_peak_filling', max_early_filling_vol),
        ('LV_late_peak_filling_ml_sec', (1000 *max_late_filling_slope)),
        ('LV_time_late_peak_filling', (max_late_filling_time - min_vol_time)),
        ('LV_vol_late_peak_filling', max_late_filling_vol),
        ('LV_time_fill80', (eighty_percent_time - min_vol_time)),
        ('LV_ratio_fill80', (eighty_percent_time - min_vol_time) / diastolic_time)
   ])


In [5]:
params178 = lv_params(patients[178])

In [6]:
def la_params(patient, region='LA'):
    units, time_units = 'ml', 'ms'
    patient_name = patient.pat_name
    
    volume_series = patient[region]
    time = np.asarray(volume_series.index)
    volume = np.asarray(volume_series.values) 
    diffs = np.gradient(volume) / np.diff(time)[0]
    r_to_r = time[-1]
    sg_volume     = savgol_filter(x=volume, window_length=5, polyorder=2, mode='interp')
    dt_msecs      = np.mean(np.diff(time))
    # one order higher for the derivative sounds right .. increase window to match
    sg_derivative = savgol_filter(x=volume, window_length=7, polyorder=3, deriv=1, mode='interp', delta=dt_msecs)

    spline = InterpolatedUnivariateSpline(time, sg_volume)
    spline_deriv = InterpolatedUnivariateSpline(time, sg_derivative)

    interpolated_time   = np.linspace(0, time[-1], len(time)*3)
    interpolated_savgol = spline(interpolated_time)
    interpolated_deriv  = spline_deriv(interpolated_time)

    max_vol_index = la_es_index(interpolated_savgol)
    max_vol_time  = interpolated_time[max_vol_index]
    max_volume    = interpolated_savgol[max_vol_index]

    max_filling_index = la_idx_max_systolic_up_slope(interpolated_savgol, interpolated_deriv)
    max_filling_time = interpolated_time[max_filling_index]
    max_filling_vol = interpolated_savgol[max_filling_index]
    max_filling_slope = interpolated_deriv[max_filling_index]
    
    max_early_emptying_index, max_late_emptying_index = la_idx_max_emptying_slopes(interpolated_savgol, interpolated_deriv)

    max_early_emptying_time = interpolated_time[max_early_emptying_index]
    max_early_emptying_vol = interpolated_savgol[max_early_emptying_index]
    max_early_emptying_slope = interpolated_deriv[max_early_emptying_index]    

    max_late_emptying_time = interpolated_time[max_late_emptying_index]
    max_late_emptying_vol = interpolated_savgol[max_late_emptying_index]
    max_late_emptying_slope = interpolated_deriv[max_late_emptying_index]    

    return OrderedDict([
        ('LA_time_end_systole', max_vol_time),
        ('LA_peak_filling_ml_sec', (1000 * max_filling_slope)),
        ('LA_time_peak_filling', max_filling_time),
        ('LA_vol_peak_filling', max_filling_vol),
        ('LA_early_peak_emptying_ml_sec', (-1000 *max_early_emptying_slope)),
        ('LA_time_early_peak_emptying', (max_early_emptying_time - max_vol_time)),
        ('LA_vol_early_peak_emptying', max_early_emptying_vol),
        ('LA_late_peak_emptying_ml_sec', (-1000 *max_late_emptying_slope)),
        ('LA_time_late_peak_emptying', (max_late_emptying_time - max_vol_time)),
        ('LA_vol_late_peak_emptying', max_late_emptying_vol),
    ])

In [7]:
params178.update(la_params(patients[178]))

In [8]:
def all_params(patient):
    params =lv_params(patient)
    params.update(la_params(patient))
    return params

results = {k: all_params(p) for (k, p) in patients.items()}

In [9]:
df_results = pd.DataFrame(results).T
df_results.insert(0, 'RR_ms', df_times['RR_ms'])
df_results.head(20).T

,178,179,182,183,186,187,188,191,192,198,205,209,210,211,221,225,229,233,234,238
RR_ms,674.157303,845.070423,952.380952,821.917808,983.606557,1090.909091,714.285714,845.070423,895.522388,1034.482759,909.090909,1200.000000,909.090909,909.090909,810.810811,1090.909091,810.810811,731.707317,810.810811,952.380952
LA_early_peak_emptying_ml_sec,276.998490,224.700038,163.888526,104.980410,363.414202,207.494257,278.300678,228.763436,43.300268,80.217240,155.037550,212.650316,217.521431,198.728968,279.377088,337.656010,255.651685,348.384218,82.075574,193.398376
LA_late_peak_emptying_ml_sec,158.269147,396.176009,612.500000,343.394511,434.806548,189.131014,183.611111,142.929646,553.785797,284.150132,85.009921,408.919478,204.125298,273.640623,260.848776,738.475033,589.927818,236.190197,282.533820,477.083333
LA_peak_filling_ml_sec,178.906512,270.301828,226.458333,172.075452,267.977527,131.652611,216.039106,153.127985,405.071941,159.112633,137.939513,203.735119,248.760898,289.533155,153.082400,266.172799,305.872119,230.763203,143.847765,228.958333
LA_time_early_peak_emptying,52.474947,54.815379,49.420849,106.627175,89.322109,70.761671,101.930502,65.778455,92.940702,228.145387,82.555283,77.837838,58.968059,58.968059,105.186267,84.914005,63.111760,47.462096,42.074507,61.776062
LA_time_end_systole,288.612208,405.633803,790.733591,373.195113,433.850244,424.570025,342.857143,361.781500,371.762808,456.290774,495.331695,467.027027,400.982801,377.395577,326.077429,396.265356,462.819576,370.204351,378.670562,469.498069
LA_time_late_peak_emptying,288.612208,405.633803,123.552124,415.845983,510.412051,594.398034,342.857143,361.781500,487.938685,536.812675,377.395577,684.972973,471.744472,483.538084,452.300950,651.007371,231.409788,294.264997,315.558802,444.787645
LA_time_peak_filling,78.712420,219.261515,0.000000,106.627175,255.206026,353.808354,74.131274,164.446136,0.000000,174.464119,106.142506,0.000000,176.904177,82.555283,73.630387,84.914005,305.040175,151.878708,84.149014,0.000000
LA_vol_early_peak_emptying,47.864520,94.689106,99.811614,56.778191,114.483429,61.028417,67.159955,53.118340,97.579952,76.042265,50.791489,155.843335,92.129826,74.583143,63.857972,107.407949,133.875471,69.078300,73.790791,87.798275
LA_vol_late_peak_emptying,31.811313,45.250857,73.400000,43.885714,61.742857,33.396440,37.257143,31.180729,49.539714,50.485714,33.571429,104.654571,43.165143,31.785919,37.303429,54.050571,92.074685,35.692771,53.408074,42.257143


We need some quality control - some values are dubious here: eg 182, 192, 209 ...

 - 182
   - LV
     - fails to recover to 80%
     - misses early filling as as later is steeper
   - LA
     - missidentifies ES as second peak higher
     - final slope down too steep as seems just at final point (edge behaviour of SG?)

- Need something better to distinguish early and late phases ...
  - could get multiple peaks with some sort of peak finder and choose two highest
  - will need some sort of robust peak finder ....

In [10]:
df_results[df_results['LV_ratio_fill80'] > 0.95]

,RR_ms,LA_early_peak_emptying_ml_sec,LA_late_peak_emptying_ml_sec,LA_peak_filling_ml_sec,LA_time_early_peak_emptying,LA_time_end_systole,LA_time_late_peak_emptying,LA_time_peak_filling,LA_vol_early_peak_emptying,LA_vol_late_peak_emptying,...,LV_peak_ejection_ml_sec,LV_ratio_fill80,LV_time_early_peak_filling,LV_time_end_systole,LV_time_fill80,LV_time_late_peak_filling,LV_time_peak_ejection,LV_vol_early_peak_filling,LV_vol_late_peak_filling,LV_vol_peak_ejection
182,952.380952,163.888526,612.500000,226.458333,49.420849,790.733591,123.552124,0.000000,99.811614,73.400000,...,551.558899,1.000000,605.405405,296.525097,617.760618,617.760618,135.907336,145.995928,150.742857,127.027676
183,821.917808,104.980410,343.394511,172.075452,106.627175,373.195113,415.845983,106.627175,56.778191,43.885714,...,448.831896,0.980392,522.473158,245.242503,533.135876,543.798593,117.289893,78.972851,86.200000,62.699107
240,1153.846154,51.845238,142.123016,138.531624,59.875260,493.970894,613.721414,149.688150,87.714286,67.942857,...,542.258402,1.000000,808.316008,284.407484,823.284823,823.284823,164.656965,87.011940,91.085714,67.085295
266,1276.595745,244.678488,478.315146,229.754390,115.928695,447.153537,778.378378,298.102358,108.156828,68.685714,...,496.382070,0.980392,182.173663,380.908568,828.062105,844.623347,149.051179,74.425549,145.371429,109.273580
300,1016.949153,106.065748,291.097884,153.195571,158.314246,448.557032,527.714155,145.121393,117.817181,100.742857,...,493.975274,0.958333,211.085662,343.014201,606.871278,633.256986,158.314246,79.659052,115.028571,101.227491


----